# BAGGING

In [1]:
import pandas as pd

diabetesData = pd.read_csv('diabetes.csv')

In [2]:
diabetesData.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
colNames = []

for i in diabetesData:
    if i != 'Outcome':
        colNames.append(i)
        
X = pd.read_csv('diabetes.csv', usecols=colNames)
y = diabetesData.Outcome

In [6]:
import random
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def TrainModel(model, X_train, y_train):
    model.fit(X_train, y_train)
    
def convert_to_dataframe(data, column):
    temp = []
    temp = pd.DataFrame(data, columns=column)
    return temp

def PredictBagging(dataset, y, train_size, models):
    print('n_dataset: ', len(dataset))
    X_colnames = list() # menyimpan nama attribut dari X
    for i in dataset:
        if i not in X_colnames:
            X_colnames.append(i)
            
    X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=1-train_size, random_state=5)
    
    print('n_train: ', len(X_train))
    print('n_test: ', len(X_test))
    
    # training model
    for model in models:
        X_train_bootstrap = list()
        y_train_bootstrap = list()
        for i in range(0, len(X_train)):
            index = random.randrange(0, len(X_train))
            X_temp = list()
            for j in range(0, len(X_train.columns)):
                X_temp.append(X_train.iloc[index][j])
            X_train_bootstrap.append(X_temp)
            y_train_bootstrap.append(y[index])
        X_train_bootstrap = convert_to_dataframe(X_train_bootstrap, column=X_colnames)
        y_train_bootstrap = convert_to_dataframe(y_train_bootstrap, column=['Outcome'])
        TrainModel(model, X_train_bootstrap, y_train_bootstrap)
        
    y_pred = list()
    for model in models:
        temp = model.predict(X_test)
        y_pred.append(temp)
        
    y_pred_most_voted = list()
    for i in range(0, len(X_test)):
        temp = list()
        for j in range(0, len(y_pred)):
            temp.append(y_pred[j][i])
        y_pred_most_voted.append(int(max(set(temp), key=temp.count)))
        
    print('\nAccuracy of each model:')
    for i in range (0, len(models)):
        print('Model ', i+1, ': ', accuracy_score(y_test, y_pred[i]))
    
    print('\nEnsemble accuracy: ', accuracy_score(y_test, y_pred_most_voted))
    return y_pred_most_voted

In [18]:
from sklearn.tree import DecisionTreeClassifier

model1 = DecisionTreeClassifier(criterion='gini', splitter='random', max_depth=10, random_state=10)
model2 = DecisionTreeClassifier(criterion='entropy', splitter='random', max_depth=10, random_state=10)
model3 = DecisionTreeClassifier(criterion='gini', splitter='random', max_depth=6, random_state=5)
model4 = DecisionTreeClassifier(criterion='entropy', splitter='random', max_depth=6, random_state=5)
model5 = DecisionTreeClassifier(criterion='gini', splitter='random', max_depth=15, random_state=1)
model6 = DecisionTreeClassifier(criterion='entropy', splitter='random', max_depth=15, random_state=1)
model7 = DecisionTreeClassifier(criterion='gini', splitter='random', max_depth=8, random_state=10)
model8 = DecisionTreeClassifier(criterion='entropy', splitter='random', max_depth=8, random_state=10)
model9 = DecisionTreeClassifier(criterion='gini', splitter='random', random_state=10)
model10 = DecisionTreeClassifier(criterion='entropy', splitter='random', random_state=10)
    
models = [model1, model2, model3, model4, model5, model6, model7, model8, model9, model10]

y_pred = PredictBagging(X, y, 0.7, models)

n_dataset:  768
n_train:  537
n_test:  231

Accuracy of each model:
Model  1 :  0.5800865800865801
Model  2 :  0.6623376623376623
Model  3 :  0.645021645021645
Model  4 :  0.6883116883116883
Model  5 :  0.5844155844155844
Model  6 :  0.5930735930735931
Model  7 :  0.5887445887445888
Model  8 :  0.5454545454545454
Model  9 :  0.5367965367965368
Model  10 :  0.47619047619047616

Ensemble accuracy:  0.696969696969697
